# 2 项目实操
本章将完整地进行一个项目实操，以便更加深刻理解第1章中提及的建模流程。其中，部分独立、高频、用法相对固定的模块在本章中一次性介绍完毕。
## 2.1 项目背景
在开展建模工作前，以下信息必须确认：
- 建模目标：因建模任务出自业务需求，故**商业目标**乃唯一目标。
- 预期收益：需明确商业目标的评价标准，如存在当前解决方案，需获取目标值及当下不足点；如无现有方案，应获取参考值。
- 应用方案：当模型建立完毕后，未来业务中部署和使用方案。

以上三点，共同决定了数据收集的范围、机器学习的类别，优先选择的算法，模型评价的指标及其精度、后续维护成本等。

下面任务开始：
```
假设你是一名汽车预测油耗

MPG：油耗，一加仑油跑多少英里
本文使用 Auto MPG 数据集，文章基于此数据集构建了一个用来预测70年代末到80年代初汽车燃油效率的模型。数据集提供了汽车相关描述，包含：气缸数，排量，马力以及重量等（这些统称为属性）。

属性信息：
1.英里/加仑:连续值
2.气缸:离散值
3.位移:连续值
4.马力:连续值
5.重量:连续值
6.加速度:连续值
7.模型年份:离散值
8.产地:离散值
9.汽车品牌:字符串
```


## 2.2 数据处理
- 获取[Auto MPG](https://archive.ics.uci.edu/ml/datasets/Auto+MPG)数据


In [14]:
import pandas as pd
auto_mpg = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data',header=None,sep='\s+')
auto_mpg.columns = ['mpg','cylinders','displacement','horsepower','weight','acceleration','model_year','origin','car_name']
auto_mpg.head()

,0,1,2,3,4,5,6,7,8
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


可视化数据
准备数据

## 2.3 模型设计
选择模型
进行训练
微调模型
## 2.4 模型评估

## 2.5 应用模型
保存
部署
监控
维护